## Project Name: Sleepy Head Alert
Developed By: Nishant Sharma
### Project Summary:

Objective:

The project aims to develop a machine learning model to detect individuals sleeping in a car from uploaded videos or images. The model will:

* Identify if a person is sleeping specifically in a car, not in other types of vehicles.
* Count the number of individuals sleeping in the car.
* Estimate the age of each sleeping person in the car.

Key Features:

1. Vehicle Type Specificity:

* The model will be trained to distinguish between different vehicle types and will only trigger detection if the sleeping person is in a car.

2. Multiple Person Detection:
* The model will be capable of detecting multiple people sleeping in the car, providing a count of the sleeping individuals.

3. Age Estimation:
* The model will predict the age of each detected sleeping person, giving an estimate of their age group.

Project Scope:

1. Data Collection and Annotation:

* Extensive dataset collection of images and videos containing individuals in various states (sleeping, non-sleeping) and vehicle types, with detailed annotations.

2. Model Development and Training:

* Selection and training of an appropriate deep learning model using annotated datasets.
* Application of transfer learning techniques to enhance model performance.

3. Validation and Testing:

* Rigorous validation using separate datasets to ensure model accuracy.
* Real-world testing to evaluate model performance in diverse scenarios.

4. Implementation and Integration:

* Development of an interface for image/video upload and result display.
* Implementation of a notification system for detected sleeping individuals.

5. Deployment:

* Deployment as a web or mobile application, ensuring accessibility and usability for end-users.

Potential Applications:

1. Driver Safety Monitoring:

* Fleet Management: Monitoring drivers in commercial fleets to ensure they are not sleeping while parked, enhancing safety and operational efficiency.
* Ridesharing Services: Ensuring passenger safety by detecting sleeping drivers in ridesharing vehicles during idle times.

2. Surveillance and Security:

* Parking Lots: Monitoring parking lots to detect and respond to individuals sleeping in cars, potentially identifying unauthorized or suspicious activity.
* Residential Areas: Enhancing neighborhood security by detecting sleeping individuals in parked cars.

3. Emergency Response:

* Healthcare and Social Services: Identifying homeless individuals or those in distress who may be sleeping in cars, enabling timely intervention and support.

4. Smart Cities and IoT:

* Integrated City Surveillance: Incorporating the model into smart city initiatives for enhanced surveillance and public safety measures.
* Connected Vehicles: Integrating the model with connected vehicle systems to provide real-time monitoring and alerts.

5. Insurance and Claims:

* Accident Prevention: Helping insurance companies assess the risk of drivers sleeping in cars and potentially preventing accidents by ensuring drivers are awake.
* Claims Investigation: Assisting in the investigation of insurance claims related to accidents involving parked cars.

6. Automotive Industry:

* In-Vehicle Monitoring Systems: Integrating the model into advanced driver-assistance systems (ADAS) to monitor drivers' states and enhance safety features.

By delivering accurate detection, counting, and age estimation of sleeping individuals in cars, this project can significantly contribute to various domains, improving safety, security, and operational efficiency.

Github Link:

### Problem Statement: Automated Detection and Analysis of Sleeping Individuals in Cars

In the realm of vehicle safety and surveillance, there is a critical need for systems that can accurately detect and analyze individuals sleeping in cars. This need is driven by various factors, including ensuring driver and passenger safety, enhancing security measures in public and private spaces, and providing timely interventions in emergency situations. Existing solutions lack specificity and often fail to differentiate between types of vehicles or accurately count and analyze multiple individuals. Moreover, there is a gap in the ability to estimate the age of sleeping individuals, which can provide valuable demographic insights for various applications.

### Let's Begin:

#### Code Block 1: Data Preprocessing
First, we'll load and preprocess the data from both datasets. Here's how we can do it:

In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Paths to the datasets
car_data_path = "C:\\Users\\nisha\\OneDrive\\Documents\\Car"
utkface_data_path = "C:\\Users\\nisha\\OneDrive\\Documents\\Ethnicity Dataset\\UTKFace"

# Function to load images and labels from the car dataset
def load_car_data(path):
    categories = ["Closed", "no_yawn", "Open", "yawn"]
    images = []
    labels = []
    for category in categories:
        class_num = categories.index(category)
        for img in os.listdir(os.path.join(path, category)):
            try:
                img_array = cv2.imread(os.path.join(path, category, img), cv2.IMREAD_GRAYSCALE)
                resized_img = cv2.resize(img_array, (128, 128))
                images.append(resized_img)
                labels.append(class_num)
            except Exception as e:
                print(e)
    return np.array(images), to_categorical(labels)

# Load car data
car_images, car_labels = load_car_data(car_data_path)

# Function to load UTKFace data and extract age labels
def load_utkface_data(path):
    all_images = []
    ages = []
    for img_name in os.listdir(path):
        parts = img_name.split('_')
        if len(parts) >= 4:
            age = int(parts[0])
            img_array = cv2.imread(os.path.join(path, img_name))
            resized_img = cv2.resize(img_array, (128, 128))
            all_images.append(resized_img)
            ages.append(age)
    return np.array(all_images), np.array(ages)

# Load UTKFace data
utk_images, utk_ages = load_utkface_data(utkface_data_path)


#### Code Block 2: Model Development
For this step, we'll develop a model to detect closed eyes, which will help us identify sleeping individuals in the car. We'll use a convolutional neural network (CNN), which is effective for image classification tasks. After we establish the eye detection model, we'll use the age prediction model trained on the UTKFace dataset to estimate ages.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Model architecture
def create_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(4, activation='softmax')  # Four classes: Closed, no_yawn, Open, yawn
    ])
    return model

# Compile the model
eye_model = create_model()
eye_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Reshape car images for the model and split the data
X_car = car_images.reshape(-1, 128, 128, 1) / 255.0  # Normalize the data
X_train, X_test, y_train, y_test = train_test_split(X_car, car_labels, test_size=0.2, random_state=42)

# Train the model
history = eye_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Save the model
eye_model.save("eye_detection_model.h5")


c:\Users\nisha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 8s 82ms/step - accuracy: 0.5159 - loss: 0.9793 - val_accuracy: 0.7672 - val_loss: 0.4254
Epoch 2/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 78ms/step - accuracy: 0.7678 - loss: 0.4459 - val_accuracy: 0.8155 - val_loss: 0.3709
Epoch 3/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.7883 - loss: 0.3826 - val_accuracy: 0.8569 - val_loss: 0.3151
Epoch 4/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8405 - loss: 0.3259 - val_accuracy: 0.8448 - val_loss: 0.3262
Epoch 5/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8510 - loss: 0.3075 - val_accuracy: 0.8483 - val_loss: 0.2787
Epoch 6/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.8610 - loss: 0.2724 - val_accuracy: 0.8810 - val_loss: 0.2391
Epoch 7/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 77ms/step - accuracy: 0.8816 - loss: 0.2590 - val_accuracy: 0.9052 - val_loss: 0.2215
Epoch 8/10
73/73 ━━━━━━━━━━━━━━━━━━━━ 6s 79ms/step - accuracy: 0.9091 - loss: 0.2022 - val_accuracy: 0.9172 - v

#### Code Block 3: Integrating Age Prediction
Now, let's integrate the age prediction using the pre-trained model from the UTKFace dataset. Since we don't have a pre-trained model yet

In [3]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load a pre-trained MobileNetV2 model, excluding the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Add custom layers for age prediction
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='linear')(x)  # Predicting age as a regression problem

# This is the model we will train
age_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the age model
age_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Prepare the UTKFace images and ages for training
X_utk = utk_images.astype('float32') / 255.0  # Normalize the data
X_utk_train, X_utk_test, y_utk_train, y_utk_test = train_test_split(X_utk, utk_ages, test_size=0.2, random_state=42)

# Train the age model
age_history = age_model.fit(X_utk_train, y_utk_train, epochs=10, validation_data=(X_utk_test, y_utk_test))

# Save the age model
age_model.save("age_prediction_model.keras")


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 56s 88ms/step - loss: 233.7865 - mae: 11.3228 - val_loss: 152.1389 - val_mae: 9.4745
Epoch 2/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 48s 81ms/step - loss: 136.6587 - mae: 8.7419 - val_loss: 141.0369 - val_mae: 8.8267
Epoch 3/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 49s 83ms/step - loss: 129.2094 - mae: 8.4782 - val_loss: 159.9019 - val_mae: 9.6936
Epoch 4/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 48s 82ms/step - loss: 114.6081 - mae: 7.9602 - val_loss: 138.2375 - val_mae: 8.8778
Epoch 5/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 49s 83ms/step - loss: 107.4817 - mae: 7.7270 - val_loss: 138.3454 - val_mae: 8.8644
Epoch 6/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 49s 83ms/step - loss: 108.2916 - mae: 7.7900 - val_loss: 136.5175 - val_mae: 8.4262
Epoch 7/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 48s 81ms/step - loss: 95.4920 - mae: 7.3316 - val_loss: 127.6645 - val_mae: 8.3686
Epoch 8/10
593/593 ━━━━━━━━━━━━━━━━━━━━ 48s 81ms/step - loss: 89.0360 - mae: 7.1381

#### Code Block 4: Integration and Test Prediction

In [11]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the models
eye_model = load_model("eye_detection_model.h5")
age_model = load_model("age_prediction_model.keras")

# Function to process an image and predict sleep and age
def predict_sleep_and_age(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        return "Image not found. Please check the file path."

    try:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.resize(img_gray, (128, 128))
        img_gray = np.reshape(img_gray, (1, 128, 128, 1)) / 255.0
        eye_pred = eye_model.predict(img_gray)
        
        sleep_status = np.argmax(eye_pred[0]) == 0  # Assuming 'Closed' class is index 0
    
        if sleep_status:
            img_color = cv2.resize(img, (128, 128))
            img_color = np.reshape(img_color, (1, 128, 128, 3)) / 255.0
            age_pred = age_model.predict(img_color)
            age = age_pred[0][0]
            return f"Sleeping detected. Predicted age: {age:.1f}"
        else:
            return "No sleeping detected."
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
image_path = "C:\\Users\\nisha\\OneDrive\\Documents\\Car\\Closed\\_24.jpg"
result = predict_sleep_and_age(image_path)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 832ms/step
Sleeping detected. Predicted age: 20.8


#### Code for the Tkinter GUI

In [14]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load models
eye_model = load_model("eye_detection_model.h5")
age_model = load_model("age_prediction_model.keras")

# Function to predict sleep and age
def predict_sleep_and_age(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    if img is None:
        return "Image not found. Please check the file path."
    try:
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.resize(img_gray, (128, 128))
        img_gray = np.reshape(img_gray, (1, 128, 128, 1)) / 255.0
        eye_pred = eye_model.predict(img_gray)
        sleep_status = np.argmax(eye_pred[0]) == 0  # Assuming 'Closed' class is index 0
        if sleep_status:
            img_color = cv2.resize(img, (128, 128))
            img_color = np.reshape(img_color, (1, 128, 128, 3)) / 255.0
            age_pred = age_model.predict(img_color)
            age = age_pred[0][0]
            return f"Sleeping detected. Predicted age: {age:.1f}"
        else:
            return "No sleeping detected."
    except Exception as e:
        return f"An error occurred: {e}"

# Create the main window
window = tk.Tk()
window.title("Sleep Detection in Cars")

# Function to open an image file and automatically run detection
def load_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = Image.open(file_path)
        img = img.resize((256, 256), Image.Resampling.LANCZOS)  # Use LANCZOS for high-quality downsampling
        photo = ImageTk.PhotoImage(img)
        img_label.image = photo
        img_label.configure(image=photo)
        img_label.file_path = file_path
        result = predict_sleep_and_age(file_path)  # Automatically run detection
        result_label.config(text=result)

# Layout components
load_button = tk.Button(window, text="Load Image", command=load_image)
load_button.pack(pady=10)

img_label = tk.Label(window)
img_label.pack(pady=10)

result_label = tk.Label(window, text="Results will be shown here.")
result_label.pack(pady=10)

# Start the GUI
window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 409ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 651ms/step
